<a href="https://colab.research.google.com/github/Varun-Vi/PracticeYolo/blob/main/Team_Qubit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paste the Folloing files in google drive(under My Drive Folder):
https://drive.google.com/drive/folders/19GoGm9NxndM6A20hGJfKTw91MP3dCQ9h?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/HelmetDetection


/content/drive/MyDrive/HelmetDetection


In [ ]:
!pip install opencv-python
!pip install tensorflow
!pip install imutils
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 17.3 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import os
import imutils
from tensorflow.keras.models import load_model
import easyocr


In [ ]:
%cd /content/drive/MyDrive/HelmetDetection


/content/drive/MyDrive/HelmetDetection


In [ ]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.chdir("/content/drive/MyDrive/HelmetDetection")

In [ ]:
net = cv2.dnn.readNet("yolov3-custom_7000.weights", "yolov3-custom.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [ ]:
model = load_model('helmet-nonhelmet_cnn.h5')

In [ ]:
#cap = cv2.VideoCapture('video.mp4')
#fps = cap.get(cv2.CAP_PROP_FPS)
imagePath = os.listdir('/content/drive/MyDrive/HelmetDetection/images')
print(imagePath)


['Screenshot 2023-06-19 at 11.14.54 PM.png', 'Screenshot 2023-06-19 at 11.15.05 PM.png', 'Screenshot 2023-06-19 at 11.16.35 PM.png', 'Screenshot 2023-06-19 at 11.16.49 PM.png', 'Screenshot 2023-06-19 at 11.16.58 PM.png', '1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '13.png']


In [ ]:
COLORS = [(0,255,0),(0,0,255)]


In [ ]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

In [ ]:
#fourcc = cv2.VideoWriter_fourcc(*"XVID")
#writer = cv2.VideoWriter('output.avi', fourcc, 5,(888,500))

In [ ]:
def helmet_or_nohelmet(helmet_roi):
	try:
		helmet_roi = cv2.resize(helmet_roi, (224, 224))
		helmet_roi = np.array(helmet_roi,dtype='float32')
		helmet_roi = helmet_roi.reshape(1, 224, 224, 3)
		helmet_roi = helmet_roi/255.0
		return int(model.predict(helmet_roi)[0][0])
	except:
			pass

In [ ]:
numbers = []

In [ ]:
def getNumber(img):
  k_size = 0
  gs_border = 3
  im_blurred = cv2.GaussianBlur(img, (k_size, k_size), gs_border)
  orig_img_multiplier = 1.5
  blur_subtraction_factor = -0.5
  gamma = 0
  im_better = cv2.addWeighted(img, orig_img_multiplier, im_blurred, blur_subtraction_factor, gamma)
  reader = easyocr.Reader(['en'])
  result = reader.readtext(im_better)
  numb = ""
  for i in range(len(result)):
    numb=numb+str(result[i][1])
  numbers.append(numb)
  return result




In [ ]:
ret = True
count=0
pathCount=len(imagePath)-1
while pathCount:
    #ret, img = cap.read()

    path = f"/content/drive/MyDrive/HelmetDetection/images/{imagePath[pathCount]}"
    print(path)
    img = cv2.imread(path)
    pathCount-=1
    img = imutils.resize(img, height=500)
    height, width = img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    confidences = []
    boxes = []
    classIds = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                classIds.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h = boxes[i]
            color = [int(c) for c in COLORS[classIds[i]]]
            # green --> bike
            # red --> number plate
            if classIds[i]==0: #bike
                helmet_roi = img[max(0,y):max(0,y)+max(0,h)//4,max(0,x):max(0,x)+max(0,w)]
            else: #number plate
                x_h = x-60
                y_h = y-350
                w_h = w+100
                h_h = h+100
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 7)
                if y_h>0 and x_h>0:
                    h_r = img[y_h:y_h+h_h , x_h:x_h +w_h]
                    c = helmet_or_nohelmet(h_r)
                    cv2.putText(img,['helmet','no-helmet'][c],(x,y-100),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2)
                    cv2.rectangle(img, (x_h, y_h), (x_h + w_h, y_h + h_h),(255,0,0), 10)
                    if c==1:
                        noPlate = img[y:y+h,x:x+w]
                        print(getNumber(noPlate))
                        count = count + 1
                        os.chdir("/content/drive/MyDrive/HelmetDetection/PhotosWithoutHelmets")
                        cv2.imwrite("noHelmet" + str(count) + ".jpg" , img)
                        cv2.imwrite("NumberPlate" + str(count) + ".jpg" , noPlate)
                        os.chdir("/content/drive/MyDrive/HelmetDetection")

    #writer.write(img)
    #if cv2.waitKey(1) == 27:
       # break

#writer.release()
#cap.release()
#cv2.waitKey(0)
#cv2.destroyAllWindows()




/content/drive/MyDrive/HelmetDetection/images/13.png
/content/drive/MyDrive/HelmetDetection/images/11.png
/content/drive/MyDrive/HelmetDetection/images/10.png
1/1 [==============================] - 0s 67ms/step


[]
/content/drive/MyDrive/HelmetDetection/images/9.png
/content/drive/MyDrive/HelmetDetection/images/8.png
/content/drive/MyDrive/HelmetDetection/images/7.png
/content/drive/MyDrive/HelmetDetection/images/6.png
/content/drive/MyDrive/HelmetDetection/images/5.png
/content/drive/MyDrive/HelmetDetection/images/4.png
/content/drive/MyDrive/HelmetDetection/images/3.png
/content/drive/MyDrive/HelmetDetection/images/2.png
/content/drive/MyDrive/HelmetDetection/images/1.png
/content/drive/MyDrive/HelmetDetection/images/Screenshot 2023-06-19 at 11.16.58 PM.png
/content/drive/MyDrive/HelmetDetection/images/Screenshot 2023-06-19 at 11.16.49 PM.png
/content/drive/MyDrive/HelmetDetection/images/Screenshot 2023-06-19 at 11.16.35 PM.png
/content/drive/MyDrive/HelmetDetection/images/Screenshot 2023-06-19 at 11.15.05 PM.png
1/1 [==============================] - 0s 60ms/step


[]
